<a href="https://colab.research.google.com/github/ShadowRaccoon/PrograConcurrente2023C1/blob/main/TP1_threads_Cpp_M1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **C++**

In [1]:
%%writefile matrix_mult.cpp
#include <iostream>
#include <vector>
#include <string>
#include <random>
#include <utility>
#include <thread>
#include <stdlib.h>

#define MIN_MATRIX_SIZE 5
#define MAX_MATRIX_SIZE 20
#define MIN_MATRIX_CELL_VALUE -32
#define MAX_MATRIX_CELL_VALUE 32

typedef std::vector<std::vector<int>> Matrix;

int validateArgs(int& argc, char* argv[]);
Matrix matrixGenerator(int size, bool populate = true);
Matrix secuentialMult(Matrix&, Matrix&, int size);
Matrix concurrentMult(Matrix&, Matrix&, int size);
void printMatrix(std::string alias, Matrix&);
bool compareMatrix(Matrix&, Matrix&);

int main(int argc, char* argv[])
{
    int size = validateArgs(argc, argv);

    Matrix matrix_a  = matrixGenerator(size);
    Matrix matrix_b  = matrixGenerator(size);
    Matrix matrix_cs = secuentialMult(matrix_a, matrix_b, size);
    Matrix matrix_ch = concurrentMult(matrix_a, matrix_b, size);

    printMatrix("A", matrix_a);
    printMatrix("B", matrix_b);
    printMatrix("CS", matrix_cs);
    printMatrix("CH", matrix_ch);
    std::cout << "Las matrices CS y CH son iguales? " << (compareMatrix(matrix_cs, matrix_ch) ? "Si" : "No") << std::endl;

    return EXIT_SUCCESS;
}

// Valida los argumentos y devuelve el tamaño de la matrices a crear
int validateArgs(int& argc, char* argv[])
{
    if(argc != 2){
        std::cout << "Cantidad de argumentos incorrectos" << std::endl;
        exit(EXIT_FAILURE);
    }

    int size = atoi(argv[1]);
    if(size < MIN_MATRIX_SIZE || size > MAX_MATRIX_SIZE){
        std::cout << "El tamaño de la matriz debe estar entre " << MIN_MATRIX_SIZE << " y " << MAX_MATRIX_SIZE << std::endl;
        exit(EXIT_FAILURE);
    }

    return size;
}

// Genera una matriz y las llena de datos de ser necesario
Matrix matrixGenerator(int size, bool populate)
{
    Matrix aux(size, std::vector<int>(size));

    if(populate)
    {
        std::random_device rd; // Obtenemos un número random del hardware
        std::mt19937 gen(rd()); // Generamos la semilla (seed)
        std::uniform_int_distribution<> distr(MIN_MATRIX_CELL_VALUE, MAX_MATRIX_CELL_VALUE); // Definimos el rango

        for(int i=0; i<size; ++i)
            for(int j=0; j<size; ++j)
                aux[i][j] = distr(gen);
    }

    // "Movemos" al objeto en vez de realizar una copia
    return std::move(aux);
}

// Realiza la multiplicación entre 2 matrices y devuelve una nueva de manera secuencial
Matrix secuentialMult(Matrix& a, Matrix& b, int size)
{
    Matrix aux = matrixGenerator(size, false);

    for(int i=0; i<size; ++i)
        for(int j=0; j<size; ++j)
            for(int k=0; k<size; ++k)
                aux[i][j] += a[i][k] * b[k][j];    

    // "Movemos" al objeto en vez de realizar una copia
    return std::move(aux);
}

// Realiza la multiplicación entre 2 matrices y devuelve una nueva de manera concurrente
Matrix concurrentMult(Matrix& a, Matrix& b, int size)
{
    Matrix aux = matrixGenerator(size, false);
    std::vector<std::thread> workers;

    for(int i; i<size; ++i)
    {
        workers.push_back(std::thread(
                // Función anónima o lambda que realiza la asignación y multiplicación
                [&aux, &a, &b, i, size]()
                {
                    for(int j=0; j < size; ++j)
                        for(int k=0; k<size; ++k)
                            aux[i][j] += a[i][k] * b[k][j];
                }
            )
        );
    }
    
    for(auto& worker : workers)
    {
        worker.join();
    }

    // "Movemos" al objeto en vez de realizar una copia
    return std::move(aux);
}

// Imprime la matriz con su alias
void printMatrix(std::string alias, Matrix& m)
{
    std::cout << "Matrix " << alias << ":" << std::endl;
    
    for(const auto& row : m)
    {
        for(const auto& col : row)
            std::cout << "\t" << col << "\t";
        std::cout << std::endl;
    }
}

bool compareMatrix(Matrix& a, Matrix& b)
{
    for(int i=0; i<a.size(); ++i)
        for(int j=0; j<a.size(); ++j)
            if(a[i][j] != b[i][j])
                return false;

    return true;
}


Writing matrix_mult.cpp


In [4]:
!g++ matrix_mult.cpp -pthread -std=c++14 -o matrix_mult.out

In [5]:
!./matrix_mult.out 1

El tamaño de la matriz debe estar entre 5 y 20


In [6]:
!./matrix_mult.out 5

Matrix A:
	-8		-2		4		-27		-32	
	12		-2		-2		3		24	
	26		28		-11		18		-19	
	-25		-21		7		-11		-13	
	23		1		-7		-20		12	
Matrix B:
	4		26		-25		-32		-23	
	-7		-13		32		16		-28	
	13		1		21		17		-9	
	-2		-14		-17		30		21	
	6		-29		-4		-14		-18	
Matrix CS:
	-104		1128		807		-70		213	
	174		-402		-553		-696		-571	
	-385		600		-215		235		-563	
	82		161		339		435		1103	
	106		510		-398		-1607		-1130	
Matrix CH:
	-104		1128		807		-70		213	
	174		-402		-553		-696		-571	
	-385		600		-215		235		-563	
	82		161		339		435		1103	
	106		510		-398		-1607		-1130	
Las matrices CS y CH son iguales? Si
